In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# setting project path
import os
import sys

gparent = os.path.join(os.pardir, os.pardir)
sys.path.append(gparent)

# imports
import pandas as pd
import numpy as np

## Background Information

This projects aims to create and train a predictive model on a dataset of Seattle Terry Stops Data. The initial data was obtain as a .csv file of 47,213 rows and 23 feature columns of Terry Stops Data.

Data set obtained from:
[Link](https://catalog.data.gov/dataset/terry-stops)

Data Columns Explanations:
[Link](https://data.seattle.gov/Public-Safety/Terry-Stops/28ny-9ts8)

## Data Cleaning

### Initial Steps
Loading the data, checking the info.

In [3]:
# creating path to data
path = os.path.join(gparent, 'data/raw', 'Terry_Stops.csv')
df = pd.read_csv(path)

In [4]:
# checking first few rows
df.head()

,Subject Age Group,Subject ID,GO / SC Num,Terry Stop ID,Stop Resolution,Weapon Type,Officer ID,Officer YOB,Officer Gender,Officer Race,...,Reported Time,Initial Call Type,Final Call Type,Call Type,Officer Squad,Arrest Flag,Frisk Flag,Precinct,Sector,Beat
0,-,-1,20140000120677,92317,Arrest,None,7500,1984,M,Black or African American,...,11:32:00,-,-,-,SOUTH PCT 1ST W - ROBERT,N,N,South,O,O2
1,-,-1,20150000001463,28806,Field Contact,None,5670,1965,M,White,...,07:59:00,-,-,-,NaN,N,N,-,-,-
2,-,-1,20150000001516,29599,Field Contact,None,4844,1961,M,White,...,19:12:00,-,-,-,NaN,N,-,-,-,-
3,-,-1,20150000001670,32260,Field Contact,None,7539,1963,M,White,...,04:55:00,-,-,-,NaN,N,N,-,-,-
4,-,-1,20150000001739,33155,Field Contact,None,6973,1977,M,White,...,00:41:00,-,-,-,NaN,N,N,-,-,-


In [5]:
#  checking info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47213 entries, 0 to 47212
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Subject Age Group         47213 non-null  object
 1   Subject ID                47213 non-null  int64 
 2   GO / SC Num               47213 non-null  int64 
 3   Terry Stop ID             47213 non-null  int64 
 4   Stop Resolution           47213 non-null  object
 5   Weapon Type               47213 non-null  object
 6   Officer ID                47213 non-null  object
 7   Officer YOB               47213 non-null  int64 
 8   Officer Gender            47213 non-null  object
 9   Officer Race              47213 non-null  object
 10  Subject Perceived Race    47213 non-null  object
 11  Subject Perceived Gender  47213 non-null  object
 12  Reported Date             47213 non-null  object
 13  Reported Time             47213 non-null  object
 14  Initial Call Type     

### Checking for Nulls

In [6]:
# checking for nulls
df.isna().sum()

Subject Age Group             0
Subject ID                    0
GO / SC Num                   0
Terry Stop ID                 0
Stop Resolution               0
Weapon Type                   0
Officer ID                    0
Officer YOB                   0
Officer Gender                0
Officer Race                  0
Subject Perceived Race        0
Subject Perceived Gender      0
Reported Date                 0
Reported Time                 0
Initial Call Type             0
Final Call Type               0
Call Type                     0
Officer Squad               604
Arrest Flag                   0
Frisk Flag                    0
Precinct                      0
Sector                        0
Beat                          0
dtype: int64

### Handling Nulls
It appears that there 604 null values, but there is also a placeholder value of `'-'` that we need to deal with as well. Replacing both with the string `'NA'`.

In [7]:
# filling nulls and replacing - values
df['Officer Squad'].fillna('NA', inplace=True)
df.replace('-', 'NA', inplace=True)

In [8]:
#  checking work
df.isna().sum()

Subject Age Group           0
Subject ID                  0
GO / SC Num                 0
Terry Stop ID               0
Stop Resolution             0
Weapon Type                 0
Officer ID                  0
Officer YOB                 0
Officer Gender              0
Officer Race                0
Subject Perceived Race      0
Subject Perceived Gender    0
Reported Date               0
Reported Time               0
Initial Call Type           0
Final Call Type             0
Call Type                   0
Officer Squad               0
Arrest Flag                 0
Frisk Flag                  0
Precinct                    0
Sector                      0
Beat                        0
dtype: int64

### Checking Feature Columns
Checking out the data, looking for any patterns & problems.

### Stop Resolution

In [9]:
# checking uniques
df['Stop Resolution'].unique()

array(['Arrest', 'Field Contact', 'Citation / Infraction',
       'Offense Report', 'Referred for Prosecution'], dtype=object)

### Creating the Target Feature
Creating a binary feature where a value of 1 means an arrest occurred and moving it to the front of the data frame. 

In [10]:
# creating and binarizing the target feature
df['Target'] = df['Stop Resolution']

df['Target'] = df['Target'].replace('Arrest', 1)
df['Target'] = df['Target'].map(lambda x: 0 if (x!=1) else 1)

In [11]:
# moving target to the front of the data frame
column_name = 'Target'
first_column = df.pop(column_name)
df.insert(0, column_name, first_column)

In [12]:
df['Target'].value_counts()

0    35546
1    11667
Name: Target, dtype: int64

In [13]:
df['Target'].value_counts(normalize=True)

0    0.752886
1    0.247114
Name: Target, dtype: float64

## Analysis: 24% of Stops End in Arrest

### Weapon Type

In [14]:
# checking weapon types
df['Weapon Type'].unique()

array(['None', 'Firearm Other', 'Lethal Cutting Instrument', 'Handgun',
       'NA', 'Club, Blackjack, Brass Knuckles',
       'Knife/Cutting/Stabbing Instrument', 'Other Firearm', 'Rifle',
       'Fire/Incendiary Device', 'Blunt Object/Striking Implement',
       'Firearm (unk type)', 'Mace/Pepper Spray', 'None/Not Applicable',
       'Club', 'Firearm', 'Taser/Stun Gun', 'Shotgun', 'Brass Knuckles',
       'Automatic Handgun', 'Blackjack',
       'Personal Weapons (hands, feet, etc.)'], dtype=object)

### Creating a Weapons Flag
Creating a feature that flags if a weapon was found during the stop.

In [15]:
# creating weapons flag
df['Weapon Flag'] = df['Weapon Type']

df['Weapon Flag'] = df['Weapon Flag'].replace(['None', 'NA', 'None/Not Applicable'], [0,0,0])
df['Weapon Flag'] = df['Weapon Flag'].map(lambda x: 1 if x!=0 else 0)

In [16]:
df['Weapon Flag'].value_counts(normalize=True)

0    0.940991
1    0.059009
Name: Weapon Flag, dtype: float64

## Analysis: Weapons Found in 6% of Stops

### Officer ID

In [17]:
# checking uniques
df['Officer ID'].unique()

array(['7500  ', '5670  ', '4844  ', ..., '6237  ', '6983  ', '5917  '],
      dtype=object)

### Fixing the Officer ID Column
Stripping whitespaces from the IDs.

In [18]:
# stripping whitespaces
df['Officer ID']= df['Officer ID'].apply(lambda x: x.strip())

In [19]:
len(df['Officer ID'].unique())

1214

## Analysis: 1214 Officers Reported Stops

### Officer Race

In [20]:
#  checking uniques
df['Officer Race'].unique()

array(['Black or African American', 'White', 'Hispanic or Latino',
       'Asian', 'American Indian/Alaska Native', 'Two or More Races',
       'Not Specified', 'Nat Hawaiian/Oth Pac Islander', 'Unknown'],
      dtype=object)

### Renaming Officer Race Categories for Brevity

In [21]:
# renaming categories for brevity
officer_race_list = ['Black or African American', 'White', 'Hispanic or Latino',
       'Asian', 'American Indian/Alaska Native', 'Two or More Races',
       'Not Specified', 'Nat Hawaiian/Oth Pac Islander', 'Unknown']

officer_replace_list = race_list = ['Black', 'White', 'Hispanic',
       'Asian', 'N_American', 'Multi-Racial',
       'NA', 'P_Islander', 'Unknown']

df['Officer Race'] = df['Officer Race'].replace(officer_race_list, officer_replace_list)

In [22]:
df['Officer Race'].value_counts(normalize=True)

White           0.755110
Hispanic        0.058077
Multi-Racial    0.056171
Asian           0.042975
Black           0.039820
NA              0.030775
P_Islander      0.009616
N_American      0.006714
Unknown         0.000741
Name: Officer Race, dtype: float64

## Analysis: 76% of Officers Reporting Stops Were White 

### Officer Gender

In [23]:
# checking uniques
df['Officer Gender'].unique()

array(['M', 'F', 'N'], dtype=object)

In [24]:
df['Officer Gender'].value_counts(normalize=True)

M    0.885053
F    0.114333
N    0.000614
Name: Officer Gender, dtype: float64

## Analysis: 88% of Officers Were Male

### Subject Race

In [25]:
# checking uniques
df['Subject Perceived Race'].unique()

array(['Asian', 'NA', 'White', 'Black or African American', 'Other',
       'Unknown', 'American Indian or Alaska Native', 'Hispanic',
       'Multi-Racial', 'Native Hawaiian or Other Pacific Islander'],
      dtype=object)

### Renaming Subject Race Categories for Brevity
Renaming, checking value counts and proportions.

In [26]:
# renaming categories for brevity
race_list = ['Asian', 'NA', 'White', 'Black or African American', 'Other',
       'Unknown', 'American Indian or Alaska Native', 'Hispanic',
       'Multi-Racial', 'Native Hawaiian or Other Pacific Islander']

replace_list =['Asian', 'NA', 'White', 'Black', 'Other',
       'Unknown', 'N_American', 'Hispanic',
       'Multi-Racial', 'P_Islander']

df['Subject Perceived Race'] = df['Subject Perceived Race']\
                                 .replace(race_list, replace_list)

In [27]:
print(df['Subject Perceived Race'].value_counts())
df['Subject Perceived Race'].value_counts(normalize=True)

White           23052
Black           14046
Unknown          2626
NA               1893
Hispanic         1684
Asian            1536
N_American       1359
Multi-Racial      809
Other             152
P_Islander         56
Name: Subject Perceived Race, dtype: int64


White           0.488255
Black           0.297503
Unknown         0.055620
NA              0.040095
Hispanic        0.035668
Asian           0.032533
N_American      0.028784
Multi-Racial    0.017135
Other           0.003219
P_Islander      0.001186
Name: Subject Perceived Race, dtype: float64

## Analysis: Subject Racial Breakdown

- White 49%

- Black 30%

- Unknown 6%

- NA 4%

- Hispanic 4%

- Asian 3%

- Native American 3%

- Multi-Racial 2%

- Other .3%

- Pacific Islander .1%   

### Subject Gender

In [28]:
# checking uniques
df['Subject Perceived Gender'].unique()

array(['Male', 'NA', 'Female', 'Unable to Determine', 'Unknown',
       'Gender Diverse (gender non-conforming and/or transgender)'],
      dtype=object)

### Renaming Subject Gender Categories

In [29]:
# renaming categories for brevity
gender_list = ['Male', 'NA', 'Female', 'Unable to Determine', 'Unknown',
       'Gender Diverse (gender non-conforming and/or transgender)']

gender_replace = ['Male', 'NA', 'Female', 'Undetermined', 'Unknown',
       'Gender Diverse']

df['Subject Perceived Gender'] = df['Subject Perceived Gender'].replace(gender_list, gender_replace)

In [30]:
df['Subject Perceived Gender'].value_counts(normalize=True)

Male              0.783534
Female            0.202974
Undetermined      0.006905
NA                0.005931
Unknown           0.000530
Gender Diverse    0.000127
Name: Subject Perceived Gender, dtype: float64

## Analysis: 78% of Subjects Were Male

### Date & Time Features

In [31]:
# checking date and time columns
print(df['Reported Date'][:2])
df['Reported Time'][:2]

0    2015-10-16T00:00:00
1    2015-03-19T00:00:00
Name: Reported Date, dtype: object


0    11:32:00
1    07:59:00
Name: Reported Time, dtype: object

### Converting Date & Time Columns to Datetime Types

In [32]:
# converting date & time columns to datetime types
df = df.astype({'Reported Time':'datetime64', 'Reported Date':'datetime64'})

### Creating Year, Month, DOM, DOW, and Hour Features

In [33]:
#  getting year, month, day of month, day of week, and hour features
df['Reported Year'] = df['Reported Date'].dt.year
df['Reported Month'] = df['Reported Date'].dt.month
df['Day of Month'] = df['Reported Date'].dt.day

# Monday = 0, Sunday = 6
df['Day of Week'] = df['Reported Date'].dt.weekday
df['Reported Hour'] = df['Reported Time'].dt.hour

In [34]:
# checking work
df.sort_values(by='Reported Year').head(2)

,Target,Subject Age Group,Subject ID,GO / SC Num,Terry Stop ID,Stop Resolution,Weapon Type,Officer ID,Officer YOB,Officer Gender,...,Frisk Flag,Precinct,Sector,Beat,Weapon Flag,Reported Year,Reported Month,Day of Month,Day of Week,Reported Hour
0,1,NA,-1,20140000120677,92317,Arrest,None,7500,1984,M,...,N,South,O,O2,0,2015,10,16,4,11
39233,0,46 - 55,-1,20150000218351,58128,Referred for Prosecution,None,7459,1973,M,...,Y,North,B,B1,0,2015,6,28,6,5


In [35]:
# dropping date and time columns
df.drop(['Reported Date', 'Reported Time'], axis=1, inplace=True)

### Checking Call Type Feature Value Counts

### Initial Call Feature

In [36]:
df['Initial Call Type'].value_counts()

NA                                                13162
SUSPICIOUS STOP - OFFICER INITIATED ONVIEW         3165
SUSPICIOUS PERSON, VEHICLE OR INCIDENT             2994
DISTURBANCE, MISCELLANEOUS/OTHER                   2429
ASLT - IP/JO - WITH OR W/O WPNS (NO SHOOTINGS)     2007
                                                  ...  
-ASSIGNED DUTY - STAKEOUT                             1
ALARM - RESIDENTIAL - SILENT/AUD PANIC/DURESS         1
MISSING - ADULT                                       1
PHONE - OBSCENE OR NUISANCE PHONE CALLS               1
MISSING - (ALZHEIMER, ENDANGERED, ELDERLY)            1
Name: Initial Call Type, Length: 167, dtype: int64

In [37]:
df['Initial Call Type'].value_counts(normalize=True)

NA                                                0.278779
SUSPICIOUS STOP - OFFICER INITIATED ONVIEW        0.067037
SUSPICIOUS PERSON, VEHICLE OR INCIDENT            0.063415
DISTURBANCE, MISCELLANEOUS/OTHER                  0.051448
ASLT - IP/JO - WITH OR W/O WPNS (NO SHOOTINGS)    0.042509
                                                    ...   
-ASSIGNED DUTY - STAKEOUT                         0.000021
ALARM - RESIDENTIAL - SILENT/AUD PANIC/DURESS     0.000021
MISSING - ADULT                                   0.000021
PHONE - OBSCENE OR NUISANCE PHONE CALLS           0.000021
MISSING - (ALZHEIMER, ENDANGERED, ELDERLY)        0.000021
Name: Initial Call Type, Length: 167, dtype: float64

## Analysis `NA` is Largest Category at 28% 

### Final Call Feature

In [38]:
df['Final Call Type'].value_counts()

NA                                          13162
--SUSPICIOUS CIRCUM. - SUSPICIOUS PERSON     3772
--PROWLER - TRESPASS                         3330
--DISTURBANCE - OTHER                        2719
--ASSAULTS, OTHER                            2297
                                            ...  
FIGHT - VERBAL/ORAL (NO WEAPONS)                1
PROWLER                                         1
-ASSIGNED DUTY - STAKEOUT                       1
ESCAPE - PRISONER                               1
TRAF - ELUDING POLICE                           1
Name: Final Call Type, Length: 207, dtype: int64

In [39]:
df['Final Call Type'].value_counts(normalize=True)

NA                                          0.278779
--SUSPICIOUS CIRCUM. - SUSPICIOUS PERSON    0.079893
--PROWLER - TRESPASS                        0.070531
--DISTURBANCE - OTHER                       0.057590
--ASSAULTS, OTHER                           0.048652
                                              ...   
FIGHT - VERBAL/ORAL (NO WEAPONS)            0.000021
PROWLER                                     0.000021
-ASSIGNED DUTY - STAKEOUT                   0.000021
ESCAPE - PRISONER                           0.000021
TRAF - ELUDING POLICE                       0.000021
Name: Final Call Type, Length: 207, dtype: float64

## Analysis: `NA` is Largest Category at 28%

### Checking Call Origination Feature

In [40]:
print(df['Call Type'].value_counts())
df['Call Type'].value_counts(normalize=True)

911                              21284
NA                               13162
ONVIEW                            9110
TELEPHONE OTHER, NOT 911          3309
ALARM CALL (NOT POLICE ALARM)      340
TEXT MESSAGE                         7
SCHEDULED EVENT (RECURRING)          1
Name: Call Type, dtype: int64


911                              0.450808
NA                               0.278779
ONVIEW                           0.192955
TELEPHONE OTHER, NOT 911         0.070087
ALARM CALL (NOT POLICE ALARM)    0.007201
TEXT MESSAGE                     0.000148
SCHEDULED EVENT (RECURRING)      0.000021
Name: Call Type, dtype: float64

## Analysis: Call Type
- 45% of the stops began as 911 calls.
- 28% are of unknown origin.
- 19% were officer initiated based on officer observation. 

`Call Type` seems to be a potentially important feature moving forward.

### Officer Squad Feature

In [41]:
df['Officer Squad'].value_counts()

TRAINING - FIELD TRAINING SQUAD                   5098
WEST PCT 1ST W - DAVID/MARY                       1546
WEST PCT 2ND W - D/M RELIEF                       1020
SOUTHWEST PCT 2ND W - FRANK                        970
NORTH PCT 2ND WATCH - NORTH BEATS                  885
                                                  ... 
TRAF - MOTORCYCLE UNIT - T2 SQUAD                    1
ZOLD CRIME ANALYSIS UNIT - ANALYSTS                  1
ROBBERY SQUAD B                                      1
TRAINING - LEARNING MANAGEMENT SYSTEM                1
COMM - INTERNET AND TELEPHONE REPORTING (ITRU)       1
Name: Officer Squad, Length: 173, dtype: int64

In [42]:
df['Officer Squad'].value_counts(normalize=True)

TRAINING - FIELD TRAINING SQUAD                   0.107979
WEST PCT 1ST W - DAVID/MARY                       0.032745
WEST PCT 2ND W - D/M RELIEF                       0.021604
SOUTHWEST PCT 2ND W - FRANK                       0.020545
NORTH PCT 2ND WATCH - NORTH BEATS                 0.018745
                                                    ...   
TRAF - MOTORCYCLE UNIT - T2 SQUAD                 0.000021
ZOLD CRIME ANALYSIS UNIT - ANALYSTS               0.000021
ROBBERY SQUAD B                                   0.000021
TRAINING - LEARNING MANAGEMENT SYSTEM             0.000021
COMM - INTERNET AND TELEPHONE REPORTING (ITRU)    0.000021
Name: Officer Squad, Length: 173, dtype: float64

### Focusing on the Training Squad
It seems that a fair number of stops come from the `TRAINING - FIELD TRAINING SQUAD`. Checking if there are more training squads.

In [43]:
sorted(df['Officer Squad'].unique())

['AUTO THEFT',
 'BURG/THEFT/JUV - EAST',
 'BURG/THEFT/JUV - NORTH',
 'BURG/THEFT/JUV - SOUTH',
 'BURG/THEFT/JUV - WEST',
 'CANINE - DAY SQUAD',
 'CANINE - NIGHT SQUAD',
 'CANINE - SQUAD A',
 'CANINE - SQUAD C',
 'CCI - MAJOR CRIMES TASK FORCE',
 'COMM - INTERNET AND TELEPHONE REPORTING (ITRU)',
 'COMMUNITY OUTREACH - SPECIAL PROJECTS DETAIL',
 'COMMUNITY OUTREACH - YOUTH VIOLENCE -SCHOOLS DETAIL',
 'COMMUNITY OUTREACH - YOUTH VIOLENCE-SCHOOLS DETAIL',
 'COMMUNITY OUTREACH UNIT',
 'CRG - SQUAD 81A',
 'CRG - SQUAD 81B',
 'CRG - SQUAD 81C',
 'CRG - SQUAD 81D',
 'CRG - SQUAD 81E',
 'CRG - SQUAD 82A',
 'CRG - SQUAD 82B',
 'CRG - SQUAD 82C',
 'CRG - SQUAD 82D',
 'CRG - SQUAD 82E',
 'CRISIS INTERVENTION COORDINATION SQUAD',
 'CRISIS RESPONSE SQUAD',
 'DV SQUAD A - INVESTIGATIONS',
 'DV SQUAD C - ELDER ABUSE',
 'DV SQUAD D - ORDER SERVICE',
 'EAST PCT 1ST W - E/G RELIEF (CHARLIE)',
 'EAST PCT 1ST W - EDWARD (CHARLIE)',
 'EAST PCT 1ST W - GEORGE (CHARLIE)',
 'EAST PCT 2ND W - BEATS',
 'EAST PCT

### Other Training Squads
Checking the number of stops from all of the training squads.

In [44]:
training = ['TRAINING - ADVANCED - SQUAD C',
'TRAINING - FIELD TRAINING SQUAD',
'TRAINING - LEARNING MANAGEMENT SYSTEM']

In [45]:
df['Officer Squad'].value_counts()[training]

TRAINING - ADVANCED - SQUAD C               1
TRAINING - FIELD TRAINING SQUAD          5098
TRAINING - LEARNING MANAGEMENT SYSTEM       1
Name: Officer Squad, dtype: int64

In [46]:
df['Officer Squad'].value_counts(normalize=True)

TRAINING - FIELD TRAINING SQUAD                   0.107979
WEST PCT 1ST W - DAVID/MARY                       0.032745
WEST PCT 2ND W - D/M RELIEF                       0.021604
SOUTHWEST PCT 2ND W - FRANK                       0.020545
NORTH PCT 2ND WATCH - NORTH BEATS                 0.018745
                                                    ...   
TRAF - MOTORCYCLE UNIT - T2 SQUAD                 0.000021
ZOLD CRIME ANALYSIS UNIT - ANALYSTS               0.000021
ROBBERY SQUAD B                                   0.000021
TRAINING - LEARNING MANAGEMENT SYSTEM             0.000021
COMM - INTERNET AND TELEPHONE REPORTING (ITRU)    0.000021
Name: Officer Squad, Length: 173, dtype: float64

## Analysis: Training Accounts for 11% of Stops
This is three times more than the next category.

### Binarizing the Arrest Flag

In [47]:
df['Arrest Flag'].value_counts()

N    43754
Y     3459
Name: Arrest Flag, dtype: int64

In [48]:
#  binarizing arrest flag
df['Arrest Flag'] = df['Arrest Flag'].replace('Y', 1)
df['Arrest Flag'] = df['Arrest Flag'].map(lambda x: 0 if x!=1 else 1)

In [49]:
df['Arrest Flag'].value_counts()

0    43754
1     3459
Name: Arrest Flag, dtype: int64

In [50]:
df['Target'].value_counts()

0    35546
1    11667
Name: Target, dtype: int64

In [51]:
df['Arrest Flag'].value_counts(normalize=True)

0    0.926736
1    0.073264
Name: Arrest Flag, dtype: float64

In [52]:
df['Target'].value_counts(normalize=True)

0    0.752886
1    0.247114
Name: Target, dtype: float64

## Analysis: Discrepancy Between Arrest Flag and Target
Oddly, the arrest flag counts are much lower than arrest (target) counts.

### Binarizing the Frisk Flag

In [53]:
df['Frisk Flag'].value_counts()

N     36147
Y     10588
NA      478
Name: Frisk Flag, dtype: int64

In [54]:
#  binarizing frisk flag
df['Frisk Flag'] = df['Frisk Flag'].replace('Y', 1)
df['Frisk Flag'] = df['Frisk Flag'].map(lambda x: 0 if x!=1 else 1)

In [55]:
df['Frisk Flag'].value_counts()

0    36625
1    10588
Name: Frisk Flag, dtype: int64

In [56]:
df['Frisk Flag'].value_counts(normalize=True)

0    0.77574
1    0.22426
Name: Frisk Flag, dtype: float64

### Checking the Precinct Feature

In [57]:
df['Precinct'].unique()

array(['South', 'NA', 'East', 'North', 'West', 'Southwest', 'Unknown',
       'SouthWest', 'OOJ', 'FK ERROR'], dtype=object)

In [58]:
df['Precinct'].value_counts()

West         11432
North        10385
NA            9859
East          6213
South         5653
Southwest     2320
SouthWest     1098
Unknown        200
OOJ             33
FK ERROR        20
Name: Precinct, dtype: int64

In [59]:
# fixing Southwest precinct
df['Precinct'] = df['Precinct'].replace('SouthWest', 'Southwest')

In [60]:
drop_list = ['Unknown', 'OOJ', 'FK ERROR']
df = df[~df['Precinct'].isin(drop_list)]

In [61]:
df['Precinct'].value_counts()

West         11432
North        10385
NA            9859
East          6213
South         5653
Southwest     3418
Name: Precinct, dtype: int64

In [62]:
df['Precinct'].value_counts(normalize=True)

West         0.243441
North        0.221146
NA           0.209945
East         0.132304
South        0.120379
Southwest    0.072785
Name: Precinct, dtype: float64

## Analysis: `NA` is Third Largest Category at 20%

### Checking the Sector Feature

In [63]:
df['Sector'].value_counts()

NA        9865
E         2337
M         2270
N         2191
K         1762
K         1725
B         1658
L         1639
D         1512
R         1455
F         1378
S         1348
U         1302
M         1254
O         1161
D         1144
J         1119
G         1087
C         1037
Q          967
W          941
E          925
Q          794
N          694
F          669
R          616
O          598
B          518
S          476
U          453
G          442
W          428
L          416
J          395
C          384
Name: Sector, dtype: int64

In [64]:
df['Sector'].value_counts(normalize=True)

NA        0.210072
E         0.049766
M         0.048339
N         0.046657
K         0.037521
K         0.036733
B         0.035307
L         0.034902
D         0.032198
R         0.030984
F         0.029344
S         0.028705
U         0.027726
M         0.026704
O         0.024723
D         0.024361
J         0.023829
G         0.023147
C         0.022083
Q         0.020592
W         0.020038
E         0.019698
Q         0.016908
N         0.014779
F         0.014246
R         0.013118
O         0.012734
B         0.011031
S         0.010136
U         0.009647
G         0.009412
W         0.009114
L         0.008859
J         0.008411
C         0.008177
Name: Sector, dtype: float64

## Analysis: `NA` is Largest at 21%
Four times larger than the next largest category.

### Checking the Beat Feature

In [65]:
df['Beat'].value_counts()

NA        9859
N3        1175
E2        1092
K3         964
M2         852
          ... 
C2          87
U3          85
J2          75
N1          73
99           2
Name: Beat, Length: 104, dtype: int64

In [66]:
df['Beat'].value_counts(normalize=True)

NA        0.209945
N3        0.025021
E2        0.023254
K3        0.020528
M2        0.018143
            ...   
C2        0.001853
U3        0.001810
J2        0.001597
N1        0.001555
99        0.000043
Name: Beat, Length: 104, dtype: float64

## Analysis: `NA` is largest Category at 21%
`NA` is over 8 times larger that the next largest category.

### Creating a Beat Flag
Creating a binary flag where 1 indicates that a beat was entered into the record.

In [67]:
# creating beat flag
df['Beat Flag'] = df['Beat']

df['Beat Flag'] = df['Beat Flag'].replace('NA', 0)
df['Beat Flag'] = df['Beat Flag'].map(lambda x: 1 if x!=0 else 0)

In [68]:
df['Beat Flag'].value_counts()

1    37101
0     9859
Name: Beat Flag, dtype: int64

In [69]:
df['Beat Flag'].value_counts(normalize=True)

1    0.790055
0    0.209945
Name: Beat Flag, dtype: float64

## Analysis: 21% of Stops Have no Beat Information

### Checking Work So Far

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46960 entries, 0 to 47212
Data columns (total 29 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Target                    46960 non-null  int64 
 1   Subject Age Group         46960 non-null  object
 2   Subject ID                46960 non-null  int64 
 3   GO / SC Num               46960 non-null  int64 
 4   Terry Stop ID             46960 non-null  int64 
 5   Stop Resolution           46960 non-null  object
 6   Weapon Type               46960 non-null  object
 7   Officer ID                46960 non-null  object
 8   Officer YOB               46960 non-null  int64 
 9   Officer Gender            46960 non-null  object
 10  Officer Race              46960 non-null  object
 11  Subject Perceived Race    46960 non-null  object
 12  Subject Perceived Gender  46960 non-null  object
 13  Initial Call Type         46960 non-null  object
 14  Final Call Type       

In [71]:
# checking columns
df.columns

Index(['Target', 'Subject Age Group', 'Subject ID', 'GO / SC Num',
       'Terry Stop ID', 'Stop Resolution', 'Weapon Type', 'Officer ID',
       'Officer YOB', 'Officer Gender', 'Officer Race',
       'Subject Perceived Race', 'Subject Perceived Gender',
       'Initial Call Type', 'Final Call Type', 'Call Type', 'Officer Squad',
       'Arrest Flag', 'Frisk Flag', 'Precinct', 'Sector', 'Beat',
       'Weapon Flag', 'Reported Year', 'Reported Month', 'Day of Month',
       'Day of Week', 'Reported Hour', 'Beat Flag'],
      dtype='object')

### Dropping Extraneous Columns

In [72]:
# dropping columns
df.drop(['Subject ID', 'GO / SC Num',
         'Terry Stop ID'], axis=1, inplace=True)

## Exporting Data as CSV Files

In [73]:
column_list = df.columns
excluded = ['Weapon Flag', 'Reported Year', 'Reported Month', 'Day of Month',
'Day of Week', 'Reported Hour', 'Beat Flag']
cols = [x for x in column_list if x not in excluded]

In [74]:
# dataframe of cleaned data with no engineered features besides the target
cleaned = df[cols]

In [75]:
# # exporting cleaned data w/ no new features
# path2 = os.path.join(gparent, 'data/processed', 'cleaned.csv')
# cleaned.to_csv(path2, index=False, na_rep='NA')

In [76]:
# # exporting cleaned data with new features
# path2 = os.path.join(gparent, 'data/processed', 'engineered.csv')
# df.to_csv(path2, index=False, na_rep='NA')